In [ ]:
import os
import json
import time
import glob
import cv2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import matplotlib.pyplot as plt

# Check training statistics

In [ ]:
experiment_folder = 'E:/detectron2/model/screw_v1/'

def load_json_arr(json_path):
    lines = []
    with open(json_path, 'r') as f:
        for line in f:
            lines.append(json.loads(line))
    return lines

experiment_metrics = load_json_arr(experiment_folder + '/metrics.json')

# total loss
total_loss_x = [x['iteration'] for x in experiment_metrics if 'total_loss' in x]
total_loss_y = [x['total_loss'] for x in experiment_metrics if 'total_loss' in x]

# validation loss
validation_loss_x = [x['iteration'] for x in experiment_metrics if 'validation_loss' in x]
validation_loss_y = [x['validation_loss'] for x in experiment_metrics if 'validation_loss' in x]

# get best validation loss
min_val_loss = 99999999
for idx, x in enumerate(experiment_metrics):
    if 'validation_loss' in x and x['validation_loss'] < min_val_loss:
        min_val_loss = x['validation_loss']
        i = x['iteration']
        min_val_loss_idx = idx
print('Best validation loss:')
print(f'iteration: {i}')
print(f'loss: {min_val_loss}')

plt.figure(figsize=(12, 5))
plt.plot(total_loss_x, total_loss_y)
plt.plot(validation_loss_x, validation_loss_y)
plt.scatter(i, min_val_loss, color='red', s=50)
plt.legend(['total_loss', 'validation_loss'], loc='upper right')
plt.ylim(0, 0.6)
plt.grid()
plt.show()




# AP
bbox_AP_x = [x['iteration'] for x in experiment_metrics if 'bbox/AP' in x]
bbox_AP_y = [x['bbox/AP'] for x in experiment_metrics if 'bbox/AP' in x]


# best AP
bbox_AP = bbox_AP_y[bbox_AP_x.index(i)]
print(f'bbox_AP: {bbox_AP}')

plt.figure(figsize=(12, 5))
plt.plot(bbox_AP_x, bbox_AP_y)
plt.scatter(i, bbox_AP, color='red', s=50)
plt.legend(['AP'], loc='upper right')
plt.grid()
plt.show()

# Inference on one image

In [ ]:
PARENT_FOLDER = 'E:/detectron2/model/screw_v1/'
SELECTED_MODEL = 'model_0001299.pth'
IMAGE_PATH = "E:/paloma/task17/IMG_0064.jpg"

CLASS_NAMES = [
    "screw",
]  # Add your class names here
CLASS_COLORS = [
    [255, 0, 0],  # Red for class 0
    # [0, 0, 255],  # Blue for class 1   # Uncomment if you have more classes
]

SCORE_THRESH = 0.8

In [ ]:
# Function to configure the model for inference
def setup_inference(config_path, model_weights_path, score_thresh=0.3):
    cfg = get_cfg()
    cfg.set_new_allowed(True)
    cfg.merge_from_file(config_path)
    cfg.MODEL.WEIGHTS = os.path.join(model_weights_path)
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = score_thresh  # Minimum score to keep a detection
    cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.5    # IoU threshold for NMS                                                                                                                                                                                                                                                                                                                                                                     
    return cfg

# Function to perform inference on a single image
def inference_on_image(image_path, predictor, metadata=None, score_thresh=0.7):
    img = cv2.imread(image_path)
    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    
    # Visualize results using visualizer
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1)
    out = visualizer.draw_instance_predictions(instances)
    
    result_image = cv2.cvtColor(out.get_image(), cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(20, 20))
    plt.imshow(result_image)
    plt.show()

    # Visualize by manual drawing on original image
    for i in range(len(instances.pred_boxes)):
        coords = instances.pred_boxes.tensor[i].int().numpy()

        # draw boxes + write text
        draw_color = CLASS_COLORS[instances.pred_classes[i]]
        text = CLASS_NAMES[instances.pred_classes[i]]
        
        x_start = coords[0]
        y_start = coords[1]
        cv2.rectangle(img, (x_start, y_start), (coords[2], coords[3]), draw_color, 2)
        
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = .5
        font_thickness = 1
        text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
        text_w, text_h = text_size
        cv2.rectangle(img, (x_start, y_start), (x_start + text_w, y_start + text_h + 5), (255, 255, 255), -1)
        cv2.putText(img, text, (x_start, y_start + text_h), font, font_scale, draw_color, font_thickness, cv2.LINE_AA)

    plt.figure(figsize=(20, 20))
    plt.imshow(img)
    plt.show()

# Paths to your config file and trained model
config_path = f"{PARENT_FOLDER}/config.yaml"
model_weights_path = f"{PARENT_FOLDER}/{SELECTED_MODEL}"
image_path = IMAGE_PATH


# Set up the model for inference
cfg = setup_inference(config_path, model_weights_path, score_thresh=SCORE_THRESH)
predictor = DefaultPredictor(cfg)
    
# Perform inference
inference_on_image(image_path, predictor)

# Inference on directory of images & save to output folder

In [ ]:
INPUT_FOLDER = "E:/paloma/task17/screw_head/coco/valid_coco/JPEGImages"  # Folder containing input images
OUTPUT_FOLDER = "E:/detectron2/output/screw_w_valid_0001299"  # Folder to save output images

In [ ]:
# Function to perform inference and save the results
def process_images_in_folder(input_folder, output_folder, predictor, metadata=None):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for file_name in os.listdir(input_folder):
        file_path = os.path.join(input_folder, file_name)
        if os.path.isfile(file_path) and file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            img = cv2.imread(file_path)
            st = time.time()
            outputs = predictor(img)
            instances = outputs["instances"].to("cpu")
            
            visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1)
            out = visualizer.draw_instance_predictions(instances)
            tt = time.time() - st
            print(tt)
            result_image = cv2.cvtColor(out.get_image(), cv2.COLOR_BGR2RGB)
            
            output_path = os.path.join(output_folder, f"result_{file_name}")
            cv2.imwrite(output_path, result_image)
            print(f"Saved result to {output_path}")

# Paths to your config file and trained model
config_path = f"{PARENT_FOLDER}/config.yaml"
model_weights_path = f"{PARENT_FOLDER}/{SELECTED_MODEL}"

# Set up the model for inference
cfg = setup_inference(config_path, model_weights_path, score_thresh=SCORE_THRESH)
predictor = DefaultPredictor(cfg)

# Process images in the input folder
process_images_in_folder(INPUT_FOLDER, OUTPUT_FOLDER, predictor)
